In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os
import sys
from mne.datasets import eegbci
import glob
from IPython.display import clear_output
import numpy as np
import torch
from torch import nn
import torch.optim as optim

from mne.datasets import eegbci
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from scipy import signal

import torch
import torch.cuda as cuda
import torch.nn as nn
from torch.autograd import Variable
import math

In [2]:
from common import EEG

In [3]:
# home directory + datasets folder
#path = '/content/drive/MyDrive/MNE-eegbci-data/files/eegmmidb/'
path = 'physionet.org/files/eegmmidb/1.0.0'
base_url = 'https://physionet.org/files/eegmmidb/'
# subjects = [1]
runs = [3, 4, 7, 8, 11, 12]
#runs = [4, 8, 12]
subjects = [1]#[i for i in range(1,40)]
#subjects = [1]
# runs = [6,10,14]

eeg = EEG(path, base_url, subjects, runs)
raw=eeg.data_to_raw()
print("Raw done")
# apply filter

raw=raw.notch_filter([50])
raw=raw.filter( 8,14, method='fir', verbose=20)
eeg.create_epochs()

Raw done
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1057 samples (6.606 sec)

Filtering raw data in 6 contiguous segments
Setting up band-pass filter from 8 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition b

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished


>>> Create Epochs.
Used Annotations descriptions: ['T1', 'T2']
Not setting metadata
90 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 90 events and 801 original time points ...
0 bad epochs dropped


(array([[   672,      0,      1],
        [  2000,      0,      0],
        [  3328,      0,      0],
        [  4656,      0,      1],
        [  5984,      0,      1],
        [  7312,      0,      0],
        [  8640,      0,      0],
        [  9968,      0,      1],
        [ 11296,      0,      0],
        [ 12624,      0,      1],
        [ 13952,      0,      1],
        [ 15280,      0,      0],
        [ 16608,      0,      0],
        [ 17936,      0,      1],
        [ 19264,      0,      0],
        [ 20672,      0,      1],
        [ 22000,      0,      0],
        [ 23328,      0,      0],
        [ 24656,      0,      1],
        [ 25984,      0,      1],
        [ 27312,      0,      0],
        [ 28640,      0,      1],
        [ 29968,      0,      0],
        [ 31296,      0,      1],
        [ 32624,      0,      0],
        [ 33952,      0,      0],
        [ 35280,      0,      1],
        [ 36608,      0,      0],
        [ 37936,      0,      1],
        [ 3926

In [4]:
# pick 7 channels.
#X2 = X[:, :14, :]
# = X2
X, y = eeg.get_X_y()

print(X.shape, y.shape)
X = X[:,:,:]
X.shape
# pick C3 and C4 channels.
X2 = X[:, 8:9, :] 
X3= X[:, 12:13, :]
X = np.concatenate((X2,X3), axis=1)
# = X4

print(X.shape)

(90, 64, 801) (90,)
(90, 2, 801)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)

print('train size',X_train.shape, y_train.shape)
print('Test size',X_test.shape, y_test.shape)

train size (63, 2, 801) (63,)
Test size (27, 2, 801) (27,)


In [6]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

#batch_size = 801

def create_dataloader(X, y, batch_size):
    X_tensor = torch.tensor(X).float()
    y_tensor = torch.tensor(y).long()
    dataset_tensor = TensorDataset(X_tensor, y_tensor)
    dl = torch.utils.data.DataLoader(dataset_tensor, batch_size=batch_size, shuffle=True)
    return dl


In [7]:



batch_size = X_train.shape[2]

train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)

num_step =math.ceil(len(train_loader.dataset) / batch_size)



in_channels  = X.shape[1] #C3, C4
out_channels = 64  #five is logical because we have freq= 8, 9, 10, 11, 12 that we want to capture
out_size     = 2  #left or right
kernel_size  = 5




In [8]:

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, out_size):
        super().__init__()
        #using sequential helps bind multiple operations together
        self.layer1 = nn.Sequential(
            #padding = (kernel_size - 1) / 2 = 2
            nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv1d(out_channels, out_channels*2, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels*2),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv1d(out_channels*2, out_channels, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(),
        )
        
        
        self.fc = nn.Linear(linear_shape  , out_size)
        #self.fc = nn.Linear(80064 , 2)
        
        self.drop_out = nn.Dropout(0.5)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.drop_out(out)
        out = self.layer2(out)
        out = self.drop_out(out)
        out = self.layer3(out)
        #out = self.drop_out(out)
        #out = self.layer4(out)
        #out = self.drop_out(out)

        out = out.reshape(out.size(0), -1)   #can also use .view()
        out = self.fc(out)
        return out

In [9]:
class ConvNet_2d(nn.Module):
    def __init__(self):
        super(ConvNet_2d, self).__init__()
        
        #using sequential helps bind multiple operations together
        self.layer1 = nn.Sequential(
            #padding = (kernel_size - 1) / 2 = 2
            nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(615168, 2)
        self.drop_out = nn.Dropout(0.5)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.drop_out(out)
        out = self.layer2(out)
        out = self.drop_out(out)
        out = out.reshape(out.size(0), -1)   #can also use .view()
        out = self.fc(out)
        return out

In [10]:
X_train = torch.FloatTensor(X_train)

In [11]:
fake_eeg_data = torch.randn_like(X_train) #(batch_size, channel, length)

In [12]:
        
        #using sequential helps bind multiple operations together
layer1 = nn.Sequential(
            #padding = (kernel_size - 1) / 2 = 2
            nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU()
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
layer2 = nn.Sequential(
            nn.Conv1d(out_channels, out_channels*2, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels*2),
            nn.ReLU()
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
layer3 = nn.Sequential(
            nn.Conv1d(out_channels*2, out_channels, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU()
        )
        

In [13]:
out = layer1(fake_eeg_data)
out.shape

torch.Size([63, 64, 801])

In [14]:
out = layer2(out)
out.shape

torch.Size([63, 128, 801])

In [15]:
out = layer3(out)
out.shape

torch.Size([63, 64, 801])

In [16]:
out = out.reshape(out.size(0), -1)

linear_shape = out.shape[1]
linear_shape

51264

In [17]:
fc = nn.Linear(linear_shape, out_size)
out = fc(out)
out.shape

torch.Size([63, 2])

In [18]:
import wandb
wandb.login()


wand = wandb.init(
        
      # Set the project where this run will be logged
      project="Motor-Imagery", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"physionet_CNN_1D_7ch_2class", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.000001,
      "architecture": "CNN2D",
      "dataset": "physionet",
      "epochs": 100000000,
      "weightname":"physionet_CNN_1D_7ch_2class",
      "num_step_per_epoch" : num_step, 
        
      }
    )

config = wand.config
print(config.num_step_per_epoch)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nutapol-1997. Use `wandb login --relogin` to force relogin


1


In [19]:
from common import train
net = ConvNet(in_channels, out_channels, kernel_size, out_size).cuda(0)
optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
criterion = nn.CrossEntropyLoss()


train_loss,valid_loss,train_accuracy,valid_accuracy =train(
    model = net,
    gpu_num = 0,
    train_loader = train_loader,
    test_loader = test_loader,
    optimizer = optimizer  ,
    criterion = criterion ,
    wand = wand
         )


wandb.alert(
            title='Finish',
            text=f'Finishing training',
        )

Epoch 1/100000000, Tr Loss: 0.7397, Tr Acc: 47.6190, Val Loss: 0.6937, Val Acc: 48.1481
Epoch 101/100000000, Tr Loss: 0.6978, Tr Acc: 58.7302, Val Loss: 0.6918, Val Acc: 51.8519
Epoch 201/100000000, Tr Loss: 0.6312, Tr Acc: 63.4921, Val Loss: 0.6902, Val Acc: 51.8519
Epoch 301/100000000, Tr Loss: 0.5718, Tr Acc: 73.0159, Val Loss: 0.6904, Val Acc: 48.1481
Epoch 401/100000000, Tr Loss: 0.5506, Tr Acc: 80.9524, Val Loss: 0.6920, Val Acc: 55.5556
Epoch 501/100000000, Tr Loss: 0.5066, Tr Acc: 80.9524, Val Loss: 0.6989, Val Acc: 55.5556


KeyboardInterrupt: 

### validate


In [17]:
del net
net = ConvNet().cuda(1)

In [18]:
# home directory + datasets folder
#path = '/content/drive/MyDrive/MNE-eegbci-data/files/eegmmidb/'
path = 'physionet.org/files/eegmmidb/1.0.0'
base_url = 'https://physionet.org/files/eegmmidb/'
# subjects = [1]
#runs = [3, 4, 7, 8, 11, 12]
runs = [4, 8, 12]
subjects = [i for i in range(83,84)]
#subjects = [1]
# runs = [6,10,14]

eeg = EEG(path, base_url, subjects, runs)
raw=eeg.data_to_raw()
print("Raw done")
# apply filter
freq = (1., 40.)
raw=eeg.filter(freq=freq)
#raw=eeg.data_to_raw()
print("Filter done")
#raw=eeg.raw_ica()
eeg.create_epochs()

Raw done
>>> Apply filter.
Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 529 samples (3.306 sec)

Filter done
>>> Create Epochs.
Used Annotations descriptions: ['T1', 'T2']
Not setting metadata
45 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 45 events and 641 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished


(array([[  672,     0,     0],
        [ 2000,     0,     1],
        [ 3328,     0,     1],
        [ 4656,     0,     0],
        [ 5984,     0,     0],
        [ 7312,     0,     1],
        [ 8640,     0,     1],
        [ 9968,     0,     0],
        [11296,     0,     1],
        [12624,     0,     0],
        [13952,     0,     1],
        [15280,     0,     0],
        [16608,     0,     1],
        [17936,     0,     0],
        [19264,     0,     0],
        [20672,     0,     0],
        [22000,     0,     1],
        [23328,     0,     0],
        [24656,     0,     1],
        [25984,     0,     1],
        [27312,     0,     0],
        [28640,     0,     0],
        [29968,     0,     1],
        [31296,     0,     1],
        [32624,     0,     0],
        [33952,     0,     1],
        [35280,     0,     0],
        [36608,     0,     1],
        [37936,     0,     0],
        [39264,     0,     1],
        [40672,     0,     0],
        [42000,     0,     1],
        

In [19]:
# pick 7 channels.
#X2 = X[:, :14, :]
# = X2

# pick C3 and C4 channels.

X_val, y_val = eeg.get_X_y()

X2 = X_val[:, 1:2, :] 
X3= X_val[:, 5:6, :]
X_val = np.concatenate((X2,X3), axis=1)
# = X4
#X_val = X_val[:, :,np.newaxis,:]
print(X_val.shape)




(45, 2, 641)


In [20]:
test_loader = create_dataloader(X_val, y_val, batch_size=batch_size)


In [21]:
# Train the model
num_epochs = 1

train_loss = []
valid_loss = []
train_accuracy = []
valid_accuracy = []
old_acc = 0
for epoch in range(num_epochs):

    loss = 0.0
    correct = 0
    iterations = 0
    net.eval()
    for i, (items, classes) in enumerate(test_loader):
        items = Variable(items)
        classes = Variable(classes)
        
        if cuda.is_available():
            items = items.cuda(1)
            classes = classes.cuda(1)
        
        outputs = net(items)
        loss += criterion(outputs, classes).item()
        
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == classes.data).sum()
        
        iterations += 1

    valid_loss.append(loss/iterations)
    correct_scalar = np.array([correct.clone().cpu()])[0]
    valid_accuracy.append(correct_scalar / len(test_loader.dataset) * 100.0)
print (' Val Acc: %.4f'
                   %(valid_accuracy[-1]))


 Val Acc: 48.8889


### Freeze layer

In [22]:
del net
net = ConvNet().cuda(1)

In [23]:
net.load_state_dict(torch.load('model_2class_2ch_CNN'))

<All keys matched successfully>

In [24]:
parms = net.state_dict()
parms.keys()

odict_keys(['layer1.0.weight', 'layer1.0.bias', 'layer1.1.weight', 'layer1.1.bias', 'layer1.1.running_mean', 'layer1.1.running_var', 'layer1.1.num_batches_tracked', 'layer2.0.weight', 'layer2.0.bias', 'layer2.1.weight', 'layer2.1.bias', 'layer2.1.running_mean', 'layer2.1.running_var', 'layer2.1.num_batches_tracked', 'fc.weight', 'fc.bias'])

In [25]:
for name,param in net.named_parameters():
    if param.requires_grad and 'layer1' in name:
        param.requires_grad = False
    if param.requires_grad and 'layer2' in name:
        param.requires_grad = False

In [26]:
# home directory + datasets folder
#path = '/content/drive/MyDrive/MNE-eegbci-data/files/eegmmidb/'
path = 'physionet.org/files/eegmmidb/1.0.0'
base_url = 'https://physionet.org/files/eegmmidb/'
# subjects = [1]
#runs = [3, 4, 7, 8, 11, 12]
runs = [4, 8, 12]
subjects = [i for i in range(83,84)]
#subjects = [1]
# runs = [6,10,14]

eeg = EEG(path, base_url, subjects, runs)
raw=eeg.data_to_raw()
print("Raw done")
# apply filter
freq = (1., 40.)
raw=eeg.filter(freq=freq)
#raw=eeg.data_to_raw()
print("Filter done")
#raw=eeg.raw_ica()
eeg.create_epochs()

Raw done
>>> Apply filter.
Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 529 samples (3.306 sec)

Filter done
>>> Create Epochs.
Used Annotations descriptions: ['T1', 'T2']
Not setting metadata
45 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 45 events and 641 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished


(array([[  672,     0,     0],
        [ 2000,     0,     1],
        [ 3328,     0,     1],
        [ 4656,     0,     0],
        [ 5984,     0,     0],
        [ 7312,     0,     1],
        [ 8640,     0,     1],
        [ 9968,     0,     0],
        [11296,     0,     1],
        [12624,     0,     0],
        [13952,     0,     1],
        [15280,     0,     0],
        [16608,     0,     1],
        [17936,     0,     0],
        [19264,     0,     0],
        [20672,     0,     0],
        [22000,     0,     1],
        [23328,     0,     0],
        [24656,     0,     1],
        [25984,     0,     1],
        [27312,     0,     0],
        [28640,     0,     0],
        [29968,     0,     1],
        [31296,     0,     1],
        [32624,     0,     0],
        [33952,     0,     1],
        [35280,     0,     0],
        [36608,     0,     1],
        [37936,     0,     0],
        [39264,     0,     1],
        [40672,     0,     0],
        [42000,     0,     1],
        

In [27]:
X, y = eeg.get_X_y()

print(X.shape, y.shape)

X = X[:,:,:]
X.shape

(45, 64, 641) (45,)


(45, 64, 641)

In [28]:
# pick 7 channels.
#X2 = X[:, :14, :]
# = X2

# pick C3 and C4 channels.
X2 = X[:, 1:2, :] 
X3= X[:, 5:6, :]
X = np.concatenate((X2,X3), axis=1)
# = X4
#X = X[:, :,np.newaxis,:]
print(X.shape)

(45, 2, 641)


In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

X_test, X_vail, y_test, y_vail = train_test_split(X_test, y_test, test_size=0.3)

print(X_test.shape, y_test.shape)
print(X_vail.shape, y_vail.shape)

(31, 2, 641) (31,)
(14, 2, 641) (14,)
(9, 2, 641) (9,)
(5, 2, 641) (5,)


In [30]:
train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)
num_step =math.ceil(len(train_loader.dataset) / batch_size)

In [31]:
#optimizer = optim.Adam(filter(lambda p: p.requires_grad, net.parameters()),lr=0.00001)
optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr=0.00001)
#optimizer = optim.Adam(net.parameters(), lr=0.0000001)
criterion = nn.CrossEntropyLoss()

In [35]:
import wandb
wandb.login()


wand = wandb.init(
        
      # Set the project where this run will be logged
      project="Motor-Imagery", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"physionet_CNN_2ch_2class_findtune", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.00001,
      "architecture": "CNN",
      "dataset": "Nutapol T.",
      "epochs": 1000,
      "weightname":"physionet_CNN_2ch_2class_findtune",
      "num_step_per_epoch" : num_step, 
        
      }
    )

config = wand.config
print(config.num_step_per_epoch)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


train/train_accuracy,▃▆▃▆▅▄▇▁▅▃▅▅▆▅▇▃▄▇▆▆▂▆▇▆▅▆▂▅▇▅▅▇▅█▃▄▇▇▅▆
train/train_loss,▆▅▅▃▄▄▂█▅▆▄▄▃▅▂▅▄▄▄▃▄▃▃▂▅▂▆▄▃▄▄▃▅▂▇▆▁▂▅▄
val/val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/val_loss,▂▂▃▁▁▃▄▄▄▄▅▅▅▅▅▅▅▅▅▅▅▆▆▇▇▆▆▇▇▇▇▇▇▇██▇███
train/train_accuracy,58.06451
train/train_loss,0.61564
val/val_accuracy,44.44444
val/val_loss,0.70689


1


In [36]:
from common import train
net = ConvNet().cuda(0)
optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
criterion = nn.CrossEntropyLoss()


train_loss,valid_loss,train_accuracy,valid_accuracy =train(
    model = net,
    gpu_num = 0,
    train_loader = train_loader,
    test_loader = test_loader,
    optimizer = optimizer  ,
    criterion = criterion ,
    wand = wand
         )


wandb.alert(
            title='Finish',
            text=f'Finishing training',
        )

Epoch 1/1000, Tr Loss: 0.7681, Tr Acc: 51.6129, Val Loss: 0.6944, Val Acc: 44.4444
Epoch 51/1000, Tr Loss: 0.3673, Tr Acc: 93.5484, Val Loss: 0.6978, Val Acc: 44.4444
Epoch 101/1000, Tr Loss: 0.1894, Tr Acc: 100.0000, Val Loss: 0.6939, Val Acc: 55.5556
Epoch 151/1000, Tr Loss: 0.1083, Tr Acc: 100.0000, Val Loss: 0.8272, Val Acc: 55.5556
Epoch 201/1000, Tr Loss: 0.0701, Tr Acc: 100.0000, Val Loss: 0.8570, Val Acc: 66.6667
Epoch 251/1000, Tr Loss: 0.0543, Tr Acc: 100.0000, Val Loss: 0.9011, Val Acc: 55.5556
Epoch 301/1000, Tr Loss: 0.0489, Tr Acc: 100.0000, Val Loss: 0.9500, Val Acc: 55.5556
Epoch 351/1000, Tr Loss: 0.0333, Tr Acc: 100.0000, Val Loss: 0.9691, Val Acc: 55.5556
Epoch 401/1000, Tr Loss: 0.0281, Tr Acc: 100.0000, Val Loss: 0.9662, Val Acc: 66.6667
Epoch 451/1000, Tr Loss: 0.0222, Tr Acc: 100.0000, Val Loss: 0.9656, Val Acc: 66.6667
Epoch 501/1000, Tr Loss: 0.0200, Tr Acc: 100.0000, Val Loss: 1.0178, Val Acc: 66.6667
Epoch 551/1000, Tr Loss: 0.0175, Tr Acc: 100.0000, Val Loss

### Validate Findtune

In [39]:
del net

In [40]:
net = ConvNet().cuda(1)

In [41]:
net.load_state_dict(torch.load('model_find_tune_2class_2ch'))

<All keys matched successfully>

In [42]:
test_loader = create_dataloader(X_vail, y_vail, batch_size=batch_size)


In [43]:
# Train the model
num_epochs = 1

train_loss = []
valid_loss = []
train_accuracy = []
valid_accuracy = []
old_acc = 0
for epoch in range(num_epochs):

    loss = 0.0
    correct = 0
    iterations = 0
    net.eval()
    for i, (items, classes) in enumerate(test_loader):
        items = Variable(items)
        classes = Variable(classes)
        
        if cuda.is_available():
            items = items.cuda(1)
            classes = classes.cuda(1)
        
        outputs = net(items)
        loss += criterion(outputs, classes).item()
        
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == classes.data).sum()
        
        iterations += 1

    valid_loss.append(loss/iterations)
    correct_scalar = np.array([correct.clone().cpu()])[0]
    valid_accuracy.append(correct_scalar / len(test_loader.dataset) * 100.0)
print (' Val Acc: %.4f'
                   %(valid_accuracy[-1]))


 Val Acc: 100.0000


In [44]:
y_vail

array([0, 0, 0, 1, 1])